<a id="2.1"></a>
<font color="blue" size=+2.5><b> Import Libraries </b></font>


In [31]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
%matplotlib inline
import random
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.colors import n_colors
from plotly.subplots import make_subplots
init_notebook_mode(connected=True)
import cufflinks as cf
cf.go_offline()
from wordcloud import WordCloud , ImageColorGenerator
from PIL import Image
!python3.7 -m pip install --upgrade pip
!pip install pywaffle
from pywaffle import Waffle 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.express as px

import cufflinks as cf 

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

%matplotlib inline
sns.set_style("whitegrid")
plt.style.use("fivethirtyeight")

import plotly.offline as py
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
from plotly import tools
init_notebook_mode(connected=True)  
import plotly.figure_factory as ff

import random

In [32]:
def random_colors(number_of_colors):
    color = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
                 for i in range(number_of_colors)]
    return color

<a id="2.2"></a>
<font color="blue" size=+2.5><b> Import Dataset </b></font>


In [33]:
deliveries = pd.read_csv('../input/ipl-complete-dataset-20082020/IPL Matches 2008-2020.csv')
matches = pd.read_csv('../input/ipl-complete-dataset-20082020/IPL Ball-by-Ball 2008-2020.csv')
result = pd.read_csv('/kaggle/working/out4.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/out4.csv'

Let's inspect the `matches` data before stepping into the concepts

In [34]:
print(f'Number of rows    = {len(matches)}')
print(f'Number of columns = {len(matches.columns)}')


Number of rows    = 193468
Number of columns = 18


In [35]:
matches.head()

,id,inning,over,ball,batsman,non_striker,bowler,batsman_runs,extra_runs,total_runs,non_boundary,is_wicket,dismissal_kind,player_dismissed,fielder,extras_type,batting_team,bowling_team
0,335982,1,6,5,RT Ponting,BB McCullum,AA Noffke,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
1,335982,1,6,6,BB McCullum,RT Ponting,AA Noffke,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
2,335982,1,7,1,BB McCullum,RT Ponting,Z Khan,0,0,0,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
3,335982,1,7,2,BB McCullum,RT Ponting,Z Khan,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
4,335982,1,7,3,RT Ponting,BB McCullum,Z Khan,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore


Let's inspect the `deliveries` data before stepping into the concepts

In [36]:
print(f'Number of rows    = {len(deliveries)}')
print(f'Number of columns = {len(deliveries.columns)}')


Number of rows    = 816
Number of columns = 17


In [37]:
deliveries.head()

,id,city,date,player_of_match,venue,neutral_venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,eliminator,method,umpire1,umpire2
0,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen
1,335983,Chandigarh,2008-04-19,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",0,Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,N,NaN,MR Benson,SL Shastri
2,335984,Delhi,2008-04-19,MF Maharoof,Feroz Shah Kotla,0,Delhi Daredevils,Rajasthan Royals,Rajasthan Royals,bat,Delhi Daredevils,wickets,9.0,N,NaN,Aleem Dar,GA Pratapkumar
3,335985,Mumbai,2008-04-20,MV Boucher,Wankhede Stadium,0,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,Royal Challengers Bangalore,wickets,5.0,N,NaN,SJ Davis,DJ Harper
4,335986,Kolkata,2008-04-20,DJ Hussey,Eden Gardens,0,Kolkata Knight Riders,Deccan Chargers,Deccan Chargers,bat,Kolkata Knight Riders,wickets,5.0,N,NaN,BF Bowden,K Hariharan


In [38]:
result.head(100)

NameError: name 'result' is not defined

<a id="2.2"></a>
<font color="blue" size=+2.5><b> Data Cleaning </b></font>

In [39]:
matches.isnull().sum()

id                       0
inning                   0
over                     0
ball                     0
batsman                  0
non_striker              0
bowler                   0
batsman_runs             0
extra_runs               0
total_runs               0
non_boundary             0
is_wicket                0
dismissal_kind      183973
player_dismissed    183973
fielder             186684
extras_type         183235
batting_team             0
bowling_team           191
dtype: int64

In [40]:
matches.columns

Index(['id', 'inning', 'over', 'ball', 'batsman', 'non_striker', 'bowler',
       'batsman_runs', 'extra_runs', 'total_runs', 'non_boundary', 'is_wicket',
       'dismissal_kind', 'player_dismissed', 'fielder', 'extras_type',
       'batting_team', 'bowling_team'],
      dtype='object')

In [41]:
deliveries.isnull().sum()

id                   0
city                13
date                 0
player_of_match      4
venue                0
neutral_venue        0
team1                0
team2                0
toss_winner          0
toss_decision        0
winner               4
result               4
result_margin       17
eliminator           4
method             797
umpire1              0
umpire2              0
dtype: int64

In [42]:
deliveries.columns

Index(['id', 'city', 'date', 'player_of_match', 'venue', 'neutral_venue',
       'team1', 'team2', 'toss_winner', 'toss_decision', 'winner', 'result',
       'result_margin', 'eliminator', 'method', 'umpire1', 'umpire2'],
      dtype='object')

In [43]:
print(deliveries['winner'].unique())
print(deliveries['city'].unique())

['Kolkata Knight Riders' 'Chennai Super Kings' 'Delhi Daredevils'
 'Royal Challengers Bangalore' 'Rajasthan Royals' 'Kings XI Punjab'
 'Deccan Chargers' 'Mumbai Indians' 'Pune Warriors' 'Kochi Tuskers Kerala'
 nan 'Sunrisers Hyderabad' 'Rising Pune Supergiants' 'Gujarat Lions'
 'Rising Pune Supergiant' 'Delhi Capitals']
['Bangalore' 'Chandigarh' 'Delhi' 'Mumbai' 'Kolkata' 'Jaipur' 'Hyderabad'
 'Chennai' 'Cape Town' 'Port Elizabeth' 'Durban' 'Centurion' 'East London'
 'Johannesburg' 'Kimberley' 'Bloemfontein' 'Ahmedabad' 'Cuttack' 'Nagpur'
 'Dharamsala' 'Kochi' 'Indore' 'Visakhapatnam' 'Pune' 'Raipur' 'Ranchi'
 'Abu Dhabi' nan 'Rajkot' 'Kanpur' 'Bengaluru' 'Dubai' 'Sharjah']


In [18]:
deliveries.team1.replace({'Rising Pune Supergiants' : 'Rising Pune Supergiant'},regex=True,inplace=True)
deliveries.team2.replace({'Rising Pune Supergiants' : 'Rising Pune Supergiant'},regex=True,inplace=True)
deliveries.winner.replace({'Rising Pune Supergiants' : 'Rising Pune Supergiant'},regex=True,inplace=True)
deliveries.venue.replace({'Feroz Shah Kotla Ground':'Feroz Shah Kotla',
                    'M Chinnaswamy Stadium':'M. Chinnaswamy Stadium',
                    'MA Chidambaram Stadium, Chepauk':'M.A. Chidambaram Stadium',
                     'M. A. Chidambaram Stadium':'M.A. Chidambaram Stadium',
                     'Punjab Cricket Association IS Bindra Stadium, Mohali':'Punjab Cricket Association Stadium',
                     'Punjab Cricket Association Stadium, Mohali':'Punjab Cricket Association Stadium',
                     'IS Bindra Stadium':'Punjab Cricket Association Stadium',
                    'Rajiv Gandhi International Stadium, Uppal':'Rajiv Gandhi International Stadium',
                    'Rajiv Gandhi Intl. Cricket Stadium':'Rajiv Gandhi International Stadium'},regex=True,inplace=True)

In [19]:
deliveries.replace('Bangalore','Bengaluru', inplace = True)

In [20]:
#fill missing values
deliveries['city'].fillna(deliveries['venue'], inplace = True)
deliveries['winner'].fillna(deliveries['result'], inplace = True)
deliveries['player_of_match'].fillna(deliveries['result'], inplace = True)
deliveries['umpire1'].fillna('unknown', inplace = True)
deliveries['umpire2'].fillna('unknown', inplace = True)

In [21]:
matches.head(20)

,id,inning,over,ball,batsman,non_striker,bowler,batsman_runs,extra_runs,total_runs,non_boundary,is_wicket,dismissal_kind,player_dismissed,fielder,extras_type,batting_team,bowling_team
0,335982,1,6,5,RT Ponting,BB McCullum,AA Noffke,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
1,335982,1,6,6,BB McCullum,RT Ponting,AA Noffke,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
2,335982,1,7,1,BB McCullum,RT Ponting,Z Khan,0,0,0,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
3,335982,1,7,2,BB McCullum,RT Ponting,Z Khan,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
4,335982,1,7,3,RT Ponting,BB McCullum,Z Khan,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
5,335982,1,7,4,BB McCullum,RT Ponting,Z Khan,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
6,335982,1,7,5,RT Ponting,BB McCullum,Z Khan,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
7,335982,1,7,6,BB McCullum,RT Ponting,Z Khan,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
8,335982,1,8,1,BB McCullum,RT Ponting,JH Kallis,0,0,0,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
9,335982,1,8,2,BB McCullum,RT Ponting,JH Kallis,0,0,0,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore


In [22]:
deliveries["season"] = deliveries["date"].apply(lambda x:x.split("-")[0])


<a id="4"></a>
<font color="blue" size=+2.5><b> Total number of matches till 2020</b></font>

In [23]:
deliveries["season"] = deliveries["date"].apply(lambda x:x.split("-")[0])

In [24]:
seasons_data = deliveries["season"].value_counts()
total_matches = deliveries.groupby('season')['id'].count()
# data = [go.Bar(
#     x = seasons_data.index,
#     y = seasons_data.values,
#     marker = dict(color = random_colors(len(seasons_data.index)),line=dict(color='#000000', width=1.5))
# )]

total_matches
# layout = go.Layout(
#    {
#       "title":"Total number of matches till 2020 (2008-2019)",
#    }
# )

# fig = go.Figure(data=data,layout = layout)
# iplot(fig)

season
2008    58
2009    57
2010    60
2011    73
2012    74
2013    76
2014    60
2015    59
2016    60
2017    59
2018    60
2019    60
2020    60
Name: id, dtype: int64

<a id="11"></a>
<font color="blue" size=+2.5><b>Top 10 Playes with Most Runs</b></font>

In [25]:
from pathlib import Path

In [29]:
filepath = Path('/kaggle/working/out5.csv')
matches['season'] = matches['id'].map(deliveries.set_index('id')['season'])

matches['season'] = pd.to_datetime(matches['season'], format='%Y').apply(lambda x: x.replace(month=1, day=1))

batting_tot=matches.groupby(['season','batsman']).apply(lambda x:np.sum(x['batsman_runs'])).reset_index(name='Runs')
#top_batsmen=batting_sorted[:10] 

# Add missing season for each batsman
seasons = matches['season'].unique()
batsmen = matches['batsman'].unique()
all_data = []
for season in seasons:
    for batsman in batsmen:
        data = {'season': season, 'batsman': batsman}
        all_data.append(data)
missing_df = pd.DataFrame(all_data)

# Merging the missing data for batsman in the main dataframe
batting_tot = pd.merge(missing_df, batting_tot, on=['season', 'batsman'], how='left')
batting_tot['Runs'] = batting_tot['Runs'].fillna(0)


#print(batting_tot[(batting_tot == 'AB de Villiers').any(axis=1)])

batting_tot['CUM_RUNS']= batting_tot.sort_values(by=['season'], ascending=True).groupby(['batsman'])['Runs'].cumsum()
batting_tot['Rank'] = batting_tot.groupby(['season'])['CUM_RUNS'].rank(method='dense', ascending=False).reset_index(drop=True)

#batting_sorted=batting_tot.sort_values(by=['season', 'CUM_RUNS'],ascending=[True,False]).groupby(['season']).head(50).reset_index(drop=True)



#batting_tot[(batting_tot == 'AB de Villiers').any(axis=1)]












batting_tot.to_csv(filepath, index=False)


# data = [go.Bar(
#     x = top_batsmen.batsman,
#     y = top_batsmen.Runs,
#     marker = dict(color = random_colors(10),line=dict(color='#000000', width=1.5))
# )]

# layout = go.Layout(title="Top 10 Batsmen in IPL- Seasons till 2020",
#                    xaxis=dict(title="Top 10 Batsmen"),
#                    yaxis=dict(title="Runs Scored"))

# # Create figure with all prepared data for plot
# fig = go.Figure(data=data, layout=layout)
# # Create a plot in your Python script directory with name "bar-chart.html"
# iplot(fig)

/opt/conda/lib/python3.7/site-packages/pandas/core/computation/expressions.py:68: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison

/opt/conda/lib/python3.7/site-packages/pandas/core/computation/expressions.py:68: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison

/opt/conda/lib/python3.7/site-packages/pandas/core/computation/expressions.py:68: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



,season,batsman,Runs,CUM_RUNS,Rank
107,2008-01-01,AB de Villiers,95.0,95.0,57.0
644,2009-01-01,AB de Villiers,465.0,560.0,15.0
1181,2010-01-01,AB de Villiers,111.0,671.0,24.0
1718,2011-01-01,AB de Villiers,312.0,983.0,23.0
2255,2012-01-01,AB de Villiers,319.0,1302.0,20.0
2792,2013-01-01,AB de Villiers,360.0,1662.0,20.0
3329,2014-01-01,AB de Villiers,395.0,2057.0,15.0
3866,2015-01-01,AB de Villiers,513.0,2570.0,10.0
4403,2016-01-01,AB de Villiers,687.0,3257.0,9.0
4940,2017-01-01,AB de Villiers,216.0,3473.0,9.0


In [ ]:
data = pd.DataFrame({"umpire":deliveries["umpire1"].tolist() + deliveries["umpire2"].tolist()},columns=['umpire'])
data = [go.Bar(
    x = data["umpire"].value_counts().index[:10],
    y = data["umpire"].value_counts().values[:10],
    marker = dict(color = random_colors(10),line=dict(color='#000000', width=1))
)]

layout = go.Layout(
   {
      "title":"Top 10 Umpire to feature in max number of matches",
   }
)
fig = go.Figure(data=data,layout = layout)
iplot(fig)

<a href="#top" class="btn btn-success btn-lg active" role="button" aria-pressed="true" style="color:white" data-toggle="popover" title="go to Colors">Go to TOP</a>